# PJM Hourly Energy Consumption Case

PJM Interconnection LLC (PJM) is a regional transmission organization (RTO) in the United States. It is part of the Eastern Interconnection grid operating an electric transmission system serving all or parts of Delaware, Illinois, Indiana, Kentucky, Maryland, Michigan, New Jersey, North Carolina, Ohio, Pennsylvania, Tennessee, Virginia, West Virginia, and the District of Columbia.

The hourly power consumption data comes from PJM's website and are in megawatts (MW).

### LSTM Training Step - By Sabrina Otoni da Silva - 2024/04

In [4]:
from pathlib import Path 

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, classification_report, ConfusionMatrixDisplay, accuracy_score, confusion_matrix

from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from keras.callbacks import EarlyStopping, LearningRateScheduler

import pickle

In [5]:
datapath = Path('../data/d02_intermediate')

In [6]:
df = pd.read_csv(f'{datapath}/pjme_train.csv')
df.set_index('datetime', inplace=True)
df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)

In [7]:
X_train = df[['hour', 'dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'day', 'weekofyear', 'lag1', 'lag2', 'lag3']]
y_train = df[['pjme_mw']]

In [8]:
print(X_train.shape)
print(y_train.shape)

(140215, 11)
(140215, 1)


In [9]:
scalerX = MinMaxScaler(feature_range=(0, 1))
scalerX.fit(X_train)
X_train= scalerX.transform(X_train).reshape(-1, X_train.shape[1], 1)

scalerY = MinMaxScaler(feature_range=(0, 1))
scalerY.fit(y_train)
y_train = scalerY.transform(y_train).reshape(-1, y_train.shape[1], 1)

In [10]:
print(X_train.shape)
print(y_train.shape)

(140215, 11, 1)
(140215, 1, 1)


In [11]:
lstm_model = Sequential()
lstm_model.add(LSTM(100, activation='relu', input_shape=(X_train.shape[1], 1)))
lstm_model.add(RepeatVector(X_train.shape[1]))
lstm_model.add(LSTM(100, activation='relu', return_sequences=True))
lstm_model.add(TimeDistributed(Dense(1)))
lstm_model.compile(optimizer='adam', loss='mse')

In [14]:
# es = EarlyStopping(patience=10, verbose=0, min_delta=0.001, monitor='loss', mode='auto', restore_best_weights=True)
# red_lr = LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)

# hist = lstm_model.fit(
#     x=X_train,
#     y=y_train,
#     batch_size=100,
#     shuffle=False,
#     epochs=300,
#     verbose=1,
#     callbacks=[es, red_lr]
# )

Epoch 1/300
  93/1403 [>.............................] - ETA: 1:08 - loss: nan

KeyboardInterrupt: 